In [1]:
import os 
os.mkdir('install-smp')
!cp -r /kaggle/input/download-smp/* install-smp
!pip install -q /kaggle/working/install-smp/munch-4.0.0-py2.py3-none-any.whl
!pip install -q /kaggle/working/install-smp/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4
!pip install -q /kaggle/working/install-smp/efficientnet_pytorch-0.7.1/efficientnet_pytorch-0.7.1
!pip install -q /kaggle/working/install-smp/timm-0.9.2-py3-none-any.whl
!pip install -q /kaggle/working/install-smp/segmentation_models_pytorch-0.3.3-py3-none-any.whl

In [2]:
import numpy as np
import pandas as pd

import torch, cv2
import torch.nn as nn

from glob import glob

import albumentations as A
from albumentations.pytorch import ToTensorV2

from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp

pd.set_option('display.max_colwidth', 200)

In [3]:
class CFG:
    target_size = 1
    model_name = 'Unet'
    backbone = 'resnext50_32x4d'
    in_chans = 3
    grid_size = [512,512]
    model_path = '/kaggle/input/models-smp/resnext50_32x4d_14.pt'
    
    test_aug = A.Compose([
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

In [4]:
image_paths = sorted(glob(f'/kaggle/input/blood-vessel-segmentation/test/*/images/*tif'))
test_df = pd.DataFrame({'image_path': image_paths})

test_df['dataset'] = test_df.image_path.map(lambda x: x.split('/')[-3])
test_df['slice'] = test_df.image_path.map(lambda x: x.split('/')[-1].replace('.tif', ''))
test_df['id'] = test_df.image_path.map(lambda x: x.split('/')[-3]+'_'+x.split("/")[-1][:-4])

test_df.head()

,image_path,dataset,slice,id
0,/kaggle/input/blood-vessel-segmentation/test/kidney_5/images/0000.tif,kidney_5,0000,kidney_5_0000
1,/kaggle/input/blood-vessel-segmentation/test/kidney_5/images/0001.tif,kidney_5,0001,kidney_5_0001
2,/kaggle/input/blood-vessel-segmentation/test/kidney_5/images/0002.tif,kidney_5,0002,kidney_5_0002
3,/kaggle/input/blood-vessel-segmentation/test/kidney_6/images/0000.tif,kidney_6,0000,kidney_6_0000
4,/kaggle/input/blood-vessel-segmentation/test/kidney_6/images/0001.tif,kidney_6,0001,kidney_6_0001


In [5]:
class CustomModel(nn.Module):
    def __init__(self, CFG, weight=None):
        super(CustomModel, self).__init__()
        self.model = smp.Unet(
            encoder_name = CFG.backbone,
            encoder_weights = weight,
            in_channels = CFG.in_chans, 
            classes = CFG.target_size, 
            activation = None 
        )
    def forward(self, image):
        output = self.model(image)
        return output

In [6]:
model = CustomModel(CFG)
model.load_state_dict(torch.load(CFG.model_path, 'cpu'))
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

CustomModel(
  (model): Unet(
    (encoder): ResNetEncoder(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        

In [7]:
def rle_encode(mask):
    pixel = mask.flatten()
    pixel = np.concatenate([[0], pixel, [0]])
    run = np.where(pixel[1:] != pixel[:-1])[0] + 1
    run[1::2] -= run[::2]
    rle = ' '.join(str(r) for r in run)
    if rle == '':
        rle = '1 0'
    return rle

In [8]:
class Load_Data(Dataset):
    def __init__(self, image_path, transform=None):
        super().__init__()
        
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        self.image_size = image.shape
        
        grid_size = CFG.grid_size
        num_i = self.image_size[0] // grid_size[0]
        num_j = self.image_size[1] // grid_size[1]

        grids = []
        for i in range(num_i):
            for j in range(num_j):
                grids.append(image[i*grid_size[0]:(i+1)*grid_size[0], j*grid_size[1]:(j+1)*grid_size[1]])
                
        for i in range(num_i):
            grids.append(image[i*grid_size[0]:(i+1)*grid_size[0], self.image_size[1]-grid_size[1]:])
            
        for j in range(num_j):
            grids.append(image[self.image_size[0]-grid_size[0]:, j*grid_size[1]:(j+1)*grid_size[1]])

        grids.append(image[self.image_size[0]-grid_size[0]:, self.image_size[1]-grid_size[1]:])
        
        self.grids = np.array(grids)
        self.transform = transform
        
    def __len__(self):
        return self.grids.shape[0]
    
    def __getitem__(self, idx):
        
        grid = np.expand_dims(self.grids[idx], axis=-1)
        grid = np.repeat(grid, 3, axis=-1)
        transformed = self.transform(image=grid)
            
        return transformed['image']

In [9]:
def gen_mask_merged(masks, image_size):
    
    idx, grid_size = 0, CFG.grid_size

    num_i = image_size[0] // grid_size[0]
    num_j = image_size[1] // grid_size[1]

    mask_merged = np.zeros(image_size)

    for i in range(num_i):
        for j in range(num_j):
            mask_merged[i*grid_size[0]:(i+1)*grid_size[0], j*grid_size[1]:(j+1)*grid_size[1]] = masks[idx]
            idx += 1

    for i in range(num_i):
        mask_merged[i*grid_size[0]:(i+1)*grid_size[0], image_size[1]-grid_size[1]:] = masks[idx]
        idx += 1

    for j in range(num_j):
        mask_merged[image_size[0]-grid_size[0]:, j*grid_size[1]:(j+1)*grid_size[1]] = masks[idx]
        idx += 1

    mask_merged[image_size[0]-grid_size[0]:, image_size[1]-grid_size[1]:] = masks[idx]

    return mask_merged

In [10]:
rles = []

with torch.no_grad():
    for _, row in test_df.iterrows():
        
        test_load = Load_Data(row['image_path'], transform=CFG.test_aug)
        test_loader = DataLoader(test_load, batch_size=32, shuffle=False)
        image_size = test_load.image_size 

        masks = np.empty(shape=(0,1,*CFG.grid_size))
        for i, grids in enumerate(test_loader):
            masks_pred = model(grids.to(device))
            masks = np.concatenate((masks, masks_pred.detach().cpu().numpy()), axis=0)
            
        masks = 1/(1 + np.exp(-masks))
        masks = (masks > 0.5)
        
        mask_merged = gen_mask_merged(masks, image_size)
        
        rles.append(rle_encode(mask_merged))
        
test_df['rle'] = rles

In [11]:
sub = test_df[['id', 'rle']]
sub.to_csv("submission.csv", index=False)
sub

,id,rle
0,kidney_5_0000,1 0
1,kidney_5_0001,1 0
2,kidney_5_0002,1 0
3,kidney_6_0000,1 0
4,kidney_6_0001,1 0
5,kidney_6_0002,1 0
